<a href="https://colab.research.google.com/github/Athugodage/RuLawSimplification/blob/main/T5/other_T5_evals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Athugodage/RuLawSimplification.git

Cloning into 'RuLawSimplification'...
remote: Enumerating objects: 710, done.
remote: Counting objects: 100% (444/444), done.
remote: Compressing objects: 100% (280/280), done.
remote: Total 710 (delta 211), reused 276 (delta 143), pack-reused 266
Receiving objects: 100% (710/710), 243.55 MiB | 18.47 MiB/s, done.
Resolving deltas: 100% (324/324), done.


In [2]:
!unzip '/content/RuLawSimplification/corpus/the_corpus.zip'

Archive:  /content/RuLawSimplification/corpus/the_corpus.zip
  inflating: the_corpus.csv          


In [3]:

import pandas as pd

df = pd.read_csv('the_corpus.csv')

In [4]:
df.columns

Index(['Название документа', 'Ссылка', 'Текст', 'Комментарий РГ', 'Date'], dtype='object')

In [5]:
df = df.drop(columns=['Название документа', 'Ссылка','Date'])

In [6]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, random_state=42, test_size=0.05)

In [7]:
train.columns = ["text", "commentary"]
test.columns = ["text", "commentary"]

In [8]:
!pip install transformers rouge_score evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 90.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 118.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 81.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8

In [9]:
from transformers import pipeline

checkpoint = 'IlyaGusev/rut5_base_sum_gazeta'
summarizer = pipeline('summarization', model=checkpoint, tokenizer=checkpoint)

In [10]:
from tqdm import tqdm


predicted = []

for text in tqdm(test['text']):
    if len(text) > 3000:
      text = text[:3000]
      
    summarized = summarizer([text])[0]
    predicted.append(summarized['summary_text'])

100%|██████████| 102/102 [55:20<00:00, 32.55s/it]


In [13]:
import evaluate

rouge = evaluate.load("rouge")
rouge.compute(references=test['text'].to_list(), predictions=[str(p) for p in predicted])

{'rouge1': 0.028391617168790698,
 'rouge2': 0.016234728260228176,
 'rougeL': 0.026481145245153766,
 'rougeLsum': 0.026261230065290866}

In [15]:
!pip install sacrebleu sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 49.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=174563c5e0d379d573a1ade310e0274685e988aa4aaac8d79e0e1dd57d6ac6de
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses


In [16]:
sari = evaluate.load("sari")

real = [list(orig) for orig in test['text']]


sari.compute(sources=test['text'].to_list(),  ## original
             references=[[ref] for ref in test['commentary'].to_list()], ## the comment from RG
             predictions=[str(p) for p in predicted]  ## model's version
             )

{'sari': 35.51942786467448}

In [18]:
!pip install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.0 MB/s eta 0:00:00


In [19]:
bertscore = evaluate.load("bertscore")

results = bertscore.compute(predictions=[str(p) for p in predicted], 
                            references=[[ref] for ref in test['commentary'].to_list()], lang='ru')
     

In [22]:
from statistics import mean
print('Precision: ', mean(results['precision']),
                          'F1:', mean(results['f1']))

Precision:  0.673158294429966 F1: 0.6332572651844398


In [26]:
import nltk

nltk.download('punkt')
## GUNNING FOG READABILITY INDEX

def gunning_fog(text):
  text = str(text)

  tokens = nltk.word_tokenize(text)
  tokens_length = len(tokens)
  sent_length = len(nltk.sent_tokenize(text))
  complex_tokens_len = len([tok for tok in tokens if len(tok) > 6])
  formule = 0.4 * (tokens_length / sent_length) + 100 * (complex_tokens_len / tokens_length)
  return formule

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [28]:
indexes = []

## original

for t in test['text'].to_list():
  indexes.append(gunning_fog(t))

print('Original text readability:', mean(indexes))


indexes = []

## simplified by model

for pred in predicted:
  indexes.append(gunning_fog(pred))

print('Summarized text readability:', mean(indexes))

Original text readability: 53.29907835008921
Summarized text readability: 62.706282784757974


In [30]:
!pip install razdel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
from razdel import sentenize, tokenize
import numpy as np


def all_syl_number(string):
    num_vowels = 0

    for char in string.lower():
        if char in "яыуаеиоюэ":
            num_vowels = num_vowels+1
       
    return num_vowels

def total_words(string):
    return len(list(tokenize(string)))

def total_sents(string):
    return len(list(sentenize(string)))
    
def fkgl(string):
    n_of_sents = total_sents(string)
    n_of_words = total_words(string)
    n_of_syls = all_syl_number(string)
    fkgl = (0.39 * (n_of_words/n_of_sents) + 11.8 * (n_of_syls/n_of_words)) - 15.39
    return float(fkgl)

# FKGL
print('FKGL')

indexes = []

## original

for t in test['text'].to_list():
  indexes.append(fkgl(t))

print('Original text readability:', np.mean(indexes))

indexes = []

## simplified by model

for pred in predicted:
  indexes.append(fkgl(pred))

print('Summarized text readability:', np.mean(indexes))

FKGL
Original text readability: 26.582805927512325
Summarized text readability: 29.049184920070907
